In [83]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [1]:
!pip install vaderSentiment

import pandas as pd
import regex as re 
import nltk

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords

nltk.download('vader_lexicon')
nltk.download('stopwords')

     |████████████████████████████████| 133kB 7.4MB/s 
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


True

# Preprocessing Lyrics

In [14]:
df = pd.read_csv('1986-2020_songs_2.csv', encoding='utf-8')
for song in range(len(df)):
  orig = df['LYRICS'][song]

  
  orig = orig.replace('\u2005', ' ')
  orig = orig.replace('\u205f', ' ')
  orig = orig.replace('\u200a', ' ')


  # remove capitlization
  orig = orig.lower()

  # remove punctuation
  orig = re.sub(r'\p{P}+', '', orig)

  # remove stopwords
  orig = orig.split(' ')
  filtered = [word for word in orig if word not in stopwords.words('english') and len(word) > 1 and word not in ['na', 'la']]
  filtered = " ".join(filtered)

  # remove identifiers like verse, chorus, etc.
  df['LYRICS'][song] = re.sub(r'[\(\[].*?[\)\]]', '', filtered)


df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,RANK,TITLE,ARTIST,YEAR,LYRICS,GENRE
0,1,That's What Friends Are For,"Dionne and Friends (Dionne Warwick, Gladys Kni...",1986,never thought id feel way far im concerned im ...,Pop
1,2,"Say You, Say Me",Lionel Richie,1986,say say say always thats way say say say toget...,R&B
2,3,I Miss You,Klymaxx,1986,thought heard voice yesterday turned around sa...,Pop
3,4,On My Own,Patti LaBelle and Michael McDonald,1986,many times said forever said love would always...,R&B
4,5,Broken Wings,Mr. Mister,1986,baby dont understand cant hold others hands ti...,Rock
...,...,...,...,...,...,...
3236,96,More Than My Hometown,Morgan Wallen,2020,girl mamas best friends whole towns rooting us...,Country
3237,97,Lovin' on You,Luke Combs,2020,dont get wrong like bobber water hookin em ree...,Country
3238,98,Said Sum,Moneybagg Yo,2020,turn yc huh ah thought broke nigga said someth...,Rap
3239,99,Slide,H.E.R.,2020,always wearin glasses dont wanna let sucker lo...,Hip-Hop


In [15]:
song = df['LYRICS'][2600]
song.count('fuck ')

8

# Sentiment Scores for Lyrics

In [16]:
analyzer = SentimentIntensityAnalyzer()
df['COMPOUND'], df['NEG'], df['NEU'], df['POS'] = 0.0, 0.0, 0.0, 0.0
for song in range (len(df)):
  scores = analyzer.polarity_scores(df['LYRICS'][song])
  df['COMPOUND'][song] = scores.get('compound')
  df['NEG'][song] = scores.get('neg')
  df['NEU'][song] = scores.get('neu')
  df['POS'][song] = scores.get('pos')
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A va

,RANK,TITLE,ARTIST,YEAR,LYRICS,GENRE,COMPOUND,NEG,NEU,POS
0,1,That's What Friends Are For,"Dionne and Friends (Dionne Warwick, Gladys Kni...",1986,never thought id feel way far im concerned im ...,Pop,0.9954,0.114,0.474,0.413
1,2,"Say You, Say Me",Lionel Richie,1986,say say say always thats way say say say toget...,R&B,0.8555,0.105,0.707,0.188
2,3,I Miss You,Klymaxx,1986,thought heard voice yesterday turned around sa...,Pop,0.9878,0.108,0.491,0.401
3,4,On My Own,Patti LaBelle and Michael McDonald,1986,many times said forever said love would always...,R&B,0.9065,0.168,0.599,0.233
4,5,Broken Wings,Mr. Mister,1986,baby dont understand cant hold others hands ti...,Rock,0.9927,0.118,0.537,0.345
...,...,...,...,...,...,...,...,...,...,...
3236,96,More Than My Hometown,Morgan Wallen,2020,girl mamas best friends whole towns rooting us...,Country,0.9980,0.077,0.549,0.374
3237,97,Lovin' on You,Luke Combs,2020,dont get wrong like bobber water hookin em ree...,Country,0.9781,0.104,0.659,0.238
3238,98,Said Sum,Moneybagg Yo,2020,turn yc huh ah thought broke nigga said someth...,Rap,-0.9951,0.205,0.709,0.085
3239,99,Slide,H.E.R.,2020,always wearin glasses dont wanna let sucker lo...,Hip-Hop,0.7707,0.077,0.811,0.112


# Profanity Analysis

In [17]:
hip_rap = ['Hip-Hop', 'Rap']
hiphop_rap_songs = df.loc[df['GENRE'].isin(hip_rap)]
len(hiphop_rap_songs)

754

754 out of 3241 songs are labeled as either 'Hip-Hop' or 'Rap'

In [34]:
# profanity vocabulary
profanity = ['nigga ', 'shit ', 'fuck ', 'bitch ', 'niggas ', 'ass ', 'fucking ', 'fucks ', 'niggaz ', 'bitches ', 'sex ', 'dick ', 'pussy ', 'fucked ', 'motherfucker ', 'fuckin ', 'lust ', 'bum ', 'motherfuckers ', 'cock ', 'shite ', 'butt ', 'whore ', 'hoe ', 'hoes ', 'slut ', 'motherfucking ', 'piss ', 'cum ']
df[profanity] = 0 
# collect counts of each profanity word for each song
for song in range(len(df)):
  lyric = df['LYRICS'][song]
  for cuss in profanity:
    df[cuss][song] = lyric.count(cuss) 

df = df.loc[:, (df != 0).any(axis=0)] # remove columns for profanity words that don't appear in any of the songs

df['profanity total']= df.iloc[:, 10:37].sum(axis=1) # sum inidividual profanity columns into new column
df.head(15)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:659: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[k] = np.nan
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

,RANK,TITLE,ARTIST,YEAR,LYRICS,GENRE,COMPOUND,NEG,NEU,POS,nigga,shit,fuck,bitch,niggas,ass,fucking,fucks,bitches,sex,dick,pussy,fucked,motherfucker,fuckin,lust,bum,motherfuckers,cock,butt,whore,hoe,hoes,slut,motherfucking,piss,cum,profanity total
0,1,That's What Friends Are For,"Dionne and Friends (Dionne Warwick, Gladys Kni...",1986,never thought id feel way far im concerned im ...,Pop,0.9954,0.114,0.474,0.413,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,"Say You, Say Me",Lionel Richie,1986,say say say always thats way say say say toget...,R&B,0.8555,0.105,0.707,0.188,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,I Miss You,Klymaxx,1986,thought heard voice yesterday turned around sa...,Pop,0.9878,0.108,0.491,0.401,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,On My Own,Patti LaBelle and Michael McDonald,1986,many times said forever said love would always...,R&B,0.9065,0.168,0.599,0.233,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,Broken Wings,Mr. Mister,1986,baby dont understand cant hold others hands ti...,Rock,0.9927,0.118,0.537,0.345,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,6,How Will I Know,Whitney Houston,1986,theres boy know hes one dream looks eyes takes...,Pop,0.9984,0.126,0.455,0.419,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,7,Party All the Time,Eddie Murphy,1986,girl cant understand want hurt things ive done...,Pop,0.9991,0.005,0.341,0.654,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,8,Burning Heart,Survivor,1986,two worlds collide rival nations primitive cla...,Rock,-0.9531,0.273,0.573,0.154,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,9,Kyrie,Mr. Mister,1986,kyrie eleison kyrie eleison kyrie wind blows h...,Pop,-0.7895,0.083,0.890,0.027,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,10,Addicted to Love,Robert Palmer,1986,lights youre home mind heart sweats body shake...,R&B,0.9990,0.008,0.533,0.459,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [35]:
df.to_csv('song_data.csv', encoding='utf-8')